<a href="https://colab.research.google.com/github/luisArthurRodriguesDaSilva/estudo-data-science-eleicoes/blob/master/apura%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
import pprint
from matplotlib import pyplot as plt
import seaborn as sns

#Endpoint TSE com resultados atualizados - 10s:
url=  'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/se/se-c0001-e000544-r.json'

#Fazendo a requisição na URL:
def trabalhoDoJuan(url,uf): #taked on 
  re=requests.get(url).json()
  candidatos = re['cand'][:10]

  df=pd.DataFrame(candidatos)
  print(df)
  df=df[['nm','n','dvt','vap','pvap']]
  df.rename(columns={'nm':'Nome do Candidato',
                  'n':'Numero do Candidato',
                  'dvt':'Validade',
                  'vap':'Votos',
                  'pvap':'(%) Votos'}, inplace=True)

  df['Horario Atualização']=re['hg']
  df['(%) Votos']=df['(%) Votos'].str.replace(',','.')
  df['(%) Votos']=df['(%) Votos'].astype(float)
  df['Votos']=df['Votos'].astype(int)
  df['Nome do Candidato']=df['Nome do Candidato'].str.replace('D&apos;',"D'")
  df=df.sort_values(by='Votos', ascending=False)

  fig, axs = plt.subplots(ncols=1, figsize=(13,6))

  ax1 = sns.barplot(
      x="Nome do Candidato",
      y='(%) Votos',
      data=df[:10],
      palette="ch:.25",
      ax=axs)

  ax1.set(xlabel="Candidatos(as)",
          ylabel="(%) Votos",
          title=f"Apuração de resultados Eleições 2022 {uf} - Presidente - (5 mais Votados)",
          ylim=(0,70))


  # Adicionando rotulo com valores no gráfico ax1:
  for p in ax1.patches:
              ax1.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                  ha='center', va='center', fontsize=13, color='black', xytext=(2, 5),
                  textcoords='offset points')

  plt.show()

  print (f'Sessões Totais em {uf}: {re["s"]}')
  print (f'Sessões Apuradas em {uf}: {re["st"]}')
  pct=int(re['st'])/int(re['s'])*100
  print (f'Percentual de sessões apuradas em {uf}: {round(pct,4)}%')

In [2]:
BRAZIL_UFs = ["AC","AL","AM","AP","BA","CE","DF","ES","GO","MA","MT","MS","MG","PA","PB","PR","PE","PI","RJ","RN","RO","RS","RR","SC","SE","SP","TO"]
brasil_ufs = ['ac','al','am','ap','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','ro','rs','rr','sc','se','sp','to']

In [3]:
def takeData(url):
  re=requests.get(url).json()
  candidatos = re['cand'][:10]

  df=pd.DataFrame(candidatos)
  df=df[['nm','n','dvt','vap','pvap']]
  df.rename(columns={'nm':'Nome do Candidato',
                  'n':'Numero do Candidato',
                  'dvt':'Validade',
                  'vap':'Votos',
                  'pvap':'(%) Votos'}, inplace=True)
  return(df)

In [4]:
urls = list(map(lambda uf : f'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/{uf}/{uf}-c0001-e000544-r.json',brasil_ufs))
datas = list(map(lambda x: takeData(x),urls))

In [5]:
THE_DATA = {}
for i, data in enumerate(datas):
  THE_DATA[brasil_ufs[i]] = data

In [9]:
THE_DATA['mg']

,Nome do Candidato,Numero do Candidato,Validade,Votos,(%) Votos
0,CIRO GOMES,12,Válido,310324,"2,58"
1,LULA,13,Válido,5802571,"48,29"
2,PADRE KELMON,14,Válido,7309,"0,06"
3,SIMONE TEBET,15,Válido,500658,"4,17"
4,VERA,16,Válido,2612,"0,02"
5,LÉO PÉRICLES,80,Válido,5241,"0,04"
6,JAIR BOLSONARO,22,Válido,5239264,"43,60"
7,CONSTITUINTE EYMAEL,27,Válido,1588,"0,01"
8,FELIPE D&apos;AVILA,30,Válido,98526,"0,82"
9,SORAYA THRONICKE,44,Válido,44109,"0,37"
